In [ ]:
from pprint import pprint
import pandas as pd
import tools as t
import regex as re

Получение данных о скилах во всех вакансиях

In [ ]:
full = pd.read_json('deduplicated.json')['key_skills']
skills = []
for element in full:
    if len(element) > 0:
        skills = skills + element
print(len(skills))

Навыки, которые не нужно разделять

In [ ]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из', 'c++']

Разделение перечислений на отдельные навыки

In [ ]:
skills = t.split_skills(skills, not_to_split)

In [ ]:
dff = pd.DataFrame(skills)
dff['orig'] = dff[0]
dff = dff.drop(columns = 0)

Удаление нерелевантных символов

In [ ]:
dff['clean'] = dff['orig'].apply(lambda x: t.clean_text(x, not_to_split))

Токенизация

In [ ]:
from nltk import word_tokenize

In [ ]:
dff['tokenize'] = dff['clean'].apply(lambda x: word_tokenize(x, language='russian'))

Лемматизация

In [ ]:
dff['lemmatize'] = dff['tokenize'].apply(lambda x: t.lemmatization(x))

Скачивание стоп-слов для русского языка и добавление новых стоп-слов

In [ ]:
from nltk.corpus import stopwords
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = t.lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

Удаление стоп-слов

In [ ]:
dff['lemmatize'] = dff['lemmatize'].apply(lambda x: t.remove_stop_words(x, stopwords))

Составление n-грамм

In [ ]:
from nltk import ngrams

In [ ]:
dff['bigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 2)))
dff['trigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 3)))
dff['fourgrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 4)))

Получение частоты n-грамм, для которых частота >=1

In [ ]:
most_common = t.most_frequent_ngrams(dff['lemmatize'], 1)
most_common_bigrams = t.most_frequent_ngrams(dff['bigrams'], 1)
most_common_trigrams = t.most_frequent_ngrams(dff['trigrams'], 1)
most_common_fourgrams = t.most_frequent_ngrams(dff['fourgrams'], 1)

In [ ]:
pprint(most_common[0:])

Сохранение полученных n-грамм

In [ ]:
df_ngrams = pd.concat([pd.DataFrame({'unigrams': most_common}), pd.DataFrame({'bigrams': most_common_bigrams}), pd.DataFrame({'trigrams': most_common_trigrams}), pd.DataFrame({'fourgrams': most_common_fourgrams})], axis = 1)

In [ ]:
def split_skill_amount(most_common):
    ngram = []
    ngram_amount = []
    for one in most_common:
        ngram.append(str(one[0]))
        ngram_amount.append(one[1])
    return ngram, ngram_amount

In [ ]:
uni, uni_amount = split_skill_amount(most_common)
bi, bi_amount = split_skill_amount(most_common_bigrams)
tri, tri_amount = split_skill_amount(most_common_trigrams)
four, four_amount = split_skill_amount(most_common_fourgrams)

In [ ]:
unigrams = pd.DataFrame({'name': uni, 'amount': uni_amount})
bigrams = pd.DataFrame({'name': bi, 'amount': bi_amount})
trigrams = pd.DataFrame({'name': tri, 'amount': tri_amount})
fourgrams = pd.DataFrame({'name': four, 'amount': four_amount})

In [ ]:
import regex as re

считать количество во всех n+1 граммах

In [ ]:
for one in trigrams['name']:
    if fourgrams['name'].str.contains(one).any() == True:
        ind = trigrams.index[trigrams['name'] == one].tolist()[0]
        trigrams['amount'][ind] = trigrams['amount'][ind] - fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()
        if trigrams['amount'][ind] < 0:
            trigrams = trigrams[trigrams.name != one]

for one in bigrams['name']:
    if trigrams['name'].str.contains(one).any() == True:
        ind = bigrams.index[bigrams['name'] == one].tolist()[0]
        bigrams['amount'][ind] = bigrams['amount'][ind] - (trigrams[trigrams['name'].str.contains(one)]['amount'].sum() + 
                                                           fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum())
        if bigrams['amount'][ind] < 0:
            bigrams = bigrams[bigrams.name != one]

for index, one in enumerate(unigrams['name']):
    search = one
    try:
        bigrams['name'].str.contains("'" + one + "'").any()
    except:
        print(one, index)
    if bigrams['name'].str.contains("'" + one + "'").any() == True:
        ind = unigrams.index[unigrams['name'] == search].tolist()[0]
        unigrams['amount'][ind] = unigrams['amount'][ind] - (bigrams[bigrams['name'].str.contains(one)]['amount'].sum() + 
                                                             trigrams[trigrams['name'].str.contains(one)]['amount'].sum() + 
                                                             fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum())
        if unigrams['amount'][ind] < 0:
            unigrams = unigrams[unigrams.name != search]

In [ ]:
print(len(unigrams), len(bigrams), len(trigrams), len(fourgrams))

In [ ]:
skills_count = unigrams.amount.sum() + bigrams.amount.sum() + trigrams.amount.sum() + fourgrams.amount.sum()
print(skills_count)

Удаление "не навыков"

In [ ]:
to_delete_from_dict = pd.read_csv('words_to_delete_from_dict.csv')['0'].to_list()

In [ ]:
ngrams_full = pd.concat([unigrams, bigrams, trigrams, fourgrams], axis = 0, ignore_index = True)

In [ ]:
ngrams_full['name'] = ngrams_full['name'].apply(lambda x: None if x in to_delete_from_dict else x)

In [ ]:
ngrams_full.dropna(inplace = True)

In [ ]:
ngrams_full.sort_values(by = 'amount', ascending = False, inplace = True, ignore_index = True)
ngrams_full['cumperc'] = ngrams_full['amount'].cumsum()/ngrams_full['amount'].sum()*100

In [ ]:
ngrams_full = ngrams_full[ngrams_full['cumperc'] < 80]

In [ ]:
ngrams_full

In [ ]:
ngrams_full.to_excel('dict-pareto_v9.xlsx')

удаление ',()

In [ ]:
ngrams_full['name'] = ngrams_full['name'].apply(lambda x: re.sub("[\'(),]", '', x))

Сохраняем словарь

In [ ]:
ngrams_full['name'].to_excel('automatic generated dict.xlsx')

In [ ]:
ngrams_full